### Monte Carlo Simulation

J Meert 2023: Performs and monte carlo simulation of N-samples from a Fisher distribution using mean paleomagnetic pole position and kappa.  Restart the kernel.

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import csv
from cartopy.mpl.geoaxes import GeoAxes

In [ ]:
#Generate Random Samples from a given distribution

def fisher_mean_direction(lats, longs):
    x = np.sum(np.cos(np.deg2rad(lats)) * np.cos(np.deg2rad(longs)))
    y = np.sum(np.cos(np.deg2rad(lats)) * np.sin(np.deg2rad(longs)))
    z = np.sum(np.sin(np.deg2rad(lats)))
    
    mean_long = np.arctan2(y, x)
    mean_lat = np.arctan2(z, np.sqrt(x**2 + y**2))
    
    return np.rad2deg(mean_lat), np.rad2deg(mean_long)

def generate_simulation(mean_lat, mean_long, kappa, n):
    lats = np.rad2deg(stats.vonmises.rvs(kappa, 0, size=n)) + mean_lat
    longs = np.rad2deg(stats.vonmises.rvs(kappa, 0, size=n)) + mean_long
    return fisher_mean_direction(lats, longs)

def generate_fisher_distributed_poles(num_sims, mean_lat, mean_long, kappa, n):
    sim_data = []

    for _ in range(num_sims):
        avg_lat, avg_long = generate_simulation(mean_lat, mean_long, kappa, n)
        sim_data.append((avg_lat, avg_long))

    return sim_data

#Plot the distribution on an orthographic map.  Lat, long centered on Mean pole location

def plot_poles_on_orthographic_map(poles):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(central_longitude=mean_long, central_latitude=mean_lat))
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    for pole in poles:
        lat, long = pole
        ax.plot(long, lat, marker='o', markersize=5, color='red', transform=ccrs.PlateCarree())

# Add mean_lat and mean_lon as a point

        ax.plot(mean_long, mean_lat, marker='o', color='red', markersize=5, alpha=0.7, transform=ccrs.PlateCarree())

# Plot the a95 circle
       
        ax.tissot(rad_km=a95 * 111, lons=mean_long, lats=mean_lat, n_samples=100, facecolor='none', edgecolor='black', linewidth=1)


def save_poles_to_csv(poles, file_name):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        for pole in poles:
            writer.writerow(pole)

# Input Mean lat, long for the paleomagnetic pole along with kappa, a95, n=number of VGPs and sims=number of simulations desired

mean_lat, mean_long = 49.8, 110.5
kappa = 265
a95 = 3.2
n = 21
sims = 5000

simulated_poles = generate_fisher_distributed_poles(sims, mean_lat, mean_long, kappa, n)
plot_poles_on_orthographic_map(simulated_poles)

## Name file for saving figure

plt.savefig('joebop.svg')
plt.show()

## Remember to name the file for saving the simulated poles in a csv file lat,long
save_poles_to_csv(simulated_poles, 'tst2.csv')
